In [ ]:
from selenium import webdriver
import requests
import re
import pandas as pd
import plotly.graph_objects as go
import time
import plotly.express as px
import itertools as it

In [ ]:
class WebDriver:
    def __init__(self, driver):
        self.driver = driver

    def __enter__(self):
        return self.driver

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.driver.quit()

df = None
items = requests.get("https://api.slothpixel.me/api/skyblock/items").json()
bazaar_items = []
for k, v in items.items():
    if v.get('bazaar'):
        bazaar_items.append(k)

In [ ]:
for item in bazaar_items:
    print(item)
    with WebDriver(webdriver.Safari()) as wd:
        wd.get(f"https://skyblock-tool.xyz/product.php?name={item}")
        time.sleep(1)
        data = wd.page_source



    dataloc = list(re.finditer('name: .*?,', data))
    start = dataloc[-1].start()
    length = data[dataloc[-1].end():].find('</script>')
    data_iter = re.finditer('"x":(.*?),"y":(.*?)}', data[start:start + length])
    t = []
    price = []
    for m in data_iter:
        t.append(m.group(1))
        price.append(m.group(2))

    if df is not None:
        df= df.append(pd.DataFrame({'item': item, 'type': 'sell', 'time': pd.to_datetime(t, utc=True, unit='ms'),
                       'price': price}), ignore_index=True)
    else:
        df = pd.DataFrame({'item': item, 'type': 'sell', 'time': pd.to_datetime(t, utc=True, unit='ms'),
                           'price': price})

    start = dataloc[-1].start()
    length = data[dataloc[-1].end():].find('</script>')
    data_iter = re.finditer('"x":(.*?),"y":(.*?)}', data[start:start + length])

    data_iter = re.finditer('"x":(.*?),"y":(.*?)}', data[dataloc[-2].end():dataloc[-1].start()])
    t = []
    price = []
    for m in data_iter:
        t.append(m.group(1))
        price.append(m.group(2))
    df = df.append(pd.DataFrame({'item': item, 'type': 'buy', 'time': pd.to_datetime(t, utc=True, unit='ms'),
                       'price': price}), ignore_index=True)

In [23]:
#df.to_csv('./BazaarMay6.csv',index=False)
df = pd.read_csv('./BazaarMay6.csv')
df=df.astype({'item':'category', 'type':'category', 'price':'float', 'time':'datetime64'})


In [ ]:
fig_dict = {
    "data": [],
    "layout": {},
    "frames": []
}
df_grouped=df.groupby(['type','item'])
for group in df2.groups:
    df3=[]
    df3 = df2.get_group(group)
    data_dict = {
        "x": df3['time'],
        "y": df3['price'],
        "mode": "lines",
        "name": f'{group[0]} {group[1]}'
    }
    fig_dict["data"].append(data_dict)

In [ ]:
fig = go.Figure(data=fig_dict)
fig.show()

In [25]:
print(df.memory_usage())
print(df.dtypes)

Index        128
item      262824
type      128164
time     1024544
price    1024544
dtype: int64
item           category
type           category
time     datetime64[ns]
price           float64
dtype: object


In [3]:
df.drop(df.columns[0],axis=1,inplace=True)

In [76]:
df2=df.set_index(keys=['item','time','type'])
df2.sort_index(level=['item','time', 'type'], ascending=[1, 1, 0], inplace=True)

In [77]:
ix = pd.IndexSlice



In [78]:
df2.loc[('YOUNG_FRAGMENT',slice(None),'buy'),:]


price
item           time                    type              
YOUNG_FRAGMENT 2020-04-29 11:52:11.123 buy   15773.159290
               2020-04-29 12:22:11.075 buy   15795.649937
               2020-04-29 12:52:11.075 buy   15828.638519
               2020-04-29 13:22:11.075 buy   16826.804878
               2020-04-29 13:52:11.225 buy   17277.329248
...                                                   ...
               2020-05-06 09:54:49.476 buy   16071.837724
               2020-05-06 10:24:49.475 buy   16041.647740
               2020-05-06 10:54:49.460 buy   16066.439339
               2020-05-06 11:24:49.434 buy   16132.235874
               2020-05-06 11:33:49.468 buy   16219.925000

[342 rows x 1 columns]

In [98]:
df2.loc[ix[:,'2020-04':'2020-05-02','buy'],:]


price
item           time                    type              
BLAZE_ROD      2020-04-29 11:22:11.118 buy      27.932244
               2020-04-29 11:52:11.123 buy      27.710173
               2020-04-29 12:22:11.075 buy      27.444207
               2020-04-29 12:52:11.075 buy      27.350466
               2020-04-29 13:22:11.075 buy      27.449583
...                                                   ...
YOUNG_FRAGMENT 2020-05-02 21:52:11.098 buy   17217.579080
               2020-05-02 22:22:11.101 buy   17223.192185
               2020-05-02 22:52:11.075 buy   17224.816391
               2020-05-02 23:22:11.101 buy   17216.741607
               2020-05-02 23:52:11.108 buy   17219.628289

[32307 rows x 1 columns]